<a href="https://colab.research.google.com/github/aditya542/MSRB_TRADE_DATA/blob/master/msrb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!pip install webdriver-manager
!pip install fake_useragent
!pip install --upgrade gupload
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from webdriver_manager.chrome import ChromeDriverManager
from google.colab import drive
!apt-get update 
!apt install chromium-chromedriver
from fake_useragent import UserAgent
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import re,time,os,codecs,csv
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth

#make browser
ua=UserAgent()
dcap = dict(DesiredCapabilities.PHANTOMJS)
dcap["phantomjs.page.settings.userAgent"] = (ua.random)
service_args=['--ssl-protocol=any','--ignore-ssl-errors=true']
#driver = webdriver.Chrome('chromedriver.exe',desired_capabilities=dcap,service_args=service_args)
#driver = webdriver.Chrome(ChromeDriverManager().install())

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

link='https://emma.msrb.org/Search/Search.aspx'
timeout = 60

    
#visit the link
driver.get(link)

but1 = driver.find_element_by_id('ctl00_mainContentArea_disclaimerContent_yesButton')
but1.click()

time.sleep(2)

data =[]  #all rows conatining data



#list of CUSIPS to be processed
cusiplist =['914119']
    
for c in cusiplist:
        
        print('Processing for Cusip::',c)
        try:
            #enter cusip value in CUSIP field
            #select = Select(driver.find_element_by_css_selector("div.trade-search-buttons")) 
            #select.select_by_index(1)
            #cinput = driver.find_element_by_css_selector('div[class*="trade-search-buttons"]') 
            cinput = driver.find_element_by_id('cusip')
            cinput.clear()
            cinput.send_keys(c)
            
            #click the search button#by class trade-search-buttons
            #element = driver.find_element_by_css_selector('div[class*="trade-search-buttons"]')
            #webdriver.ActionChains(driver).move_to_element(element ).click(element ).perform()
            runSearch = driver.find_element_by_id('runSearchButton')
            runSearch.click()
        except NoSuchElementException as e:
            print (e)        

        time.sleep(2)
        #wait for the results to be populated
        
        try:
            #table = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.ID, 'lvSearchResults')))
            table = driver.find_element_by_id('right')
            nextPage = driver.find_element_by_id('lvSecurities_next')
            while True:
                nextPage = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.ID, 'lvSecurities_next')))
                        
                #data in table
                rows = table.find_elements_by_tag_name('tr')
                for tr in rows:
                    tds = tr.find_elements_by_tag_name('td')
                    if tds:
                        row_data = []
                        idx = 0
                        for td in tds:
                            if idx == 1 :
                                row_data.append(c)
                                idx += 1
                            else:
                                row_data.append(td.text)
                                idx += 1
                        if(row_data[0] != ''):
                            data.append(row_data)

                #print(file_header) 
                #print(data)

                #Click Next Button until it is active
                if 'disabled' in nextPage.get_attribute('class'):
                    break;
                nextPage.click()
                time.sleep(3)
            
        except Exception as e:
            continue
      
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#for i in data:
#    i.to_csv('Msrb_Results"+str(i)+".csv')
#for i,e in enumerate(data):
#    df = pd.DataFrame(e)
#    df.to_csv("drive/My Drive/"+"df"+str(i)+".csv")
# Mount your Drive to the Colab VM.
#drive.mount('/content/gdrive')
# Write the DataFrame to CSV file.
#with open('/drive/My Drive/Msrb_Results.csv', 'w') as f:
# df.to_csv(f)
#!cp Msrb_Results.csv "drive/My Drive/"     
#write csv from table
try:
    with open('/content/Msrb_Results.csv', mode='w', newline='') as issue_file:
        writer = csv.writer(issue_file, delimiter=',')
    
        writer.writerow(['State','Description * ','Coupon(%)','MaturityDate',' Principal Amount at issuance','Dated Date','Fitch',	'KBRA',	'Moody','S&P'])
        for d in data:
            writer.writerow(d)
      #  !cp Msrb_Results.csv "drive\My Drive\Capstone Financial Data w. ESG"     
        print('Msrb_Results.csv file created.')
  
except Exception as e:
    print(e)
    print('Could not create csv')

Requirement already up-to-date: gupload in /usr/local/lib/python3.6/dist-packages (1.1.0)
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-driver

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: use options instead of chrome_options


Processing for Cusip:: 914119
Msrb_Results.csv file created.


# New Section